In [20]:
import pandas as pd
import sqlite3
import os

In [21]:
db = sqlite3.connect("Monarchs2000.db")

In [22]:
df = pd.read_csv("2000 Season Summary.csv")

        #convert pandas to SQL table
df.to_sql("monarchs2000", db, if_exists="replace")

os.getcwd()



'C:\\Users\\Arink\\Desktop\\github\\CodeLou\\Monarchs\\Data'

In [23]:
schema = pd.read_sql('SELECT * FROM sqlite_master ORDER by name', db)
schema

,type,name,tbl_name,rootpage,sql
0,index,ix_monarchs2000_index,monarchs2000,4,"CREATE INDEX ""ix_monarchs2000_index""ON ""monarc..."
1,table,monarchs2000,monarchs2000,2,"CREATE TABLE ""monarchs2000"" (\n""index"" INTEGER..."


## Checking DataType

In [24]:
data_type = pd.read_sql('SELECT * FROM PRAGMA_TABLE_INFO("monarchs2000")', db)
data_type

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Tag No.,TEXT,0,None,0
2,2,Tagger,TEXT,0,None,0
3,3,"Tag City, State",TEXT,0,None,0
4,4,Tag Date,TEXT,0,None,0
5,5,Report Date,TEXT,0,None,0
6,6,Report Location,TEXT,0,None,0
7,7,Reporter,TEXT,0,None,0
8,8,Miles,REAL,0,None,0


### Drop tagger and reporter identification

In [25]:
df.drop(["Tagger", "Reporter"], axis = 1, inplace = True)
df

,Tag No.,"Tag City, State",Tag Date,Report Date,Report Location,Miles
0,613PZ,"Old Lyme, CT",09/24/00,01/26/01,"El Rosario, MICH, MX",2222.0
1,486KF,"Darlington, ONT",09/12/99,01/07/01,"El Rosario, MICH, MX",2090.0
2,067GA,"Douglas Lake, MI",08/20/00,01/07/01,"El Rosario, MICH, MX",1995.0
3,154JR,"Chincoteague, VA",09/25/99,03/09/01,"Cerro Pelon, MEXICO, MX",1969.0
4,017MS,"Cresson, PA",09/01/99,12/01/00,"El Rosario, MICH, MX",1927.0
...,...,...,...,...,...,...
290,611PF,"Rochester, MN",NaN,NaN,"Dyersburg, TN",NaN
291,260PG,"Rochester, MN",NaN,02/01/01,"El Rosario, MICH, MX",NaN
292,175PG,"Rochester, MN",NaN,02/28/01,"El Rosario, MICH, MX",NaN
293,295PG,"Rochester, MN",NaN,01/26/01,"El Rosario, MICH, MX",NaN


### Split & rename combined columns in Tagged category

In [26]:
df2 = df['Tag City, State'].str.split(', ', expand=True)
df2.columns = ['tag_city', 'Tag State']
df2


,tag_city,Tag State
0,Old Lyme,CT
1,Darlington,ONT
2,Douglas Lake,MI
3,Chincoteague,VA
4,Cresson,PA
...,...,...
290,Rochester,MN
291,Rochester,MN
292,Rochester,MN
293,Rochester,MN


### Split & Rename combined columns in Reported category

In [30]:
df3 = df['Report Location'].str.split(', ', expand=True)
df3.columns = ['report_city', 'Report State', 'Report Country']
#add_usa = df3['Report Country'].fillna('USA') decided to use .loc
df3.loc[df3['Report State'] == 'MICH', 'Report State'] = 'Michoacán'
df3

,report_city,Report State,Report Country
0,El Rosario,Michoacán,MX
1,El Rosario,Michoacán,MX
2,El Rosario,Michoacán,MX
3,Cerro Pelon,MEXICO,MX
4,El Rosario,Michoacán,MX
...,...,...,...
290,Dyersburg,TN,None
291,El Rosario,Michoacán,MX
292,El Rosario,Michoacán,MX
293,El Rosario,Michoacán,MX


### Create new table with unneeded columns dropped

In [31]:
df4 = pd.concat([df, df2, df3], axis=1)
df4.drop(['Tag City, State', 'Report Location', 'report_city', 'tag_city'], axis=1, inplace=True)
df5 = df4[['Tag No.','Tag State', 'Tag Date', 'Report State', 'Report Country', 'Report Date', 'Miles']]
new_table = df5.copy()
new_table

,Tag No.,Tag State,Tag Date,Report State,Report Country,Report Date,Miles
0,613PZ,CT,09/24/00,Michoacán,MX,01/26/01,2222.0
1,486KF,ONT,09/12/99,Michoacán,MX,01/07/01,2090.0
2,067GA,MI,08/20/00,Michoacán,MX,01/07/01,1995.0
3,154JR,VA,09/25/99,MEXICO,MX,03/09/01,1969.0
4,017MS,PA,09/01/99,Michoacán,MX,12/01/00,1927.0
...,...,...,...,...,...,...,...
290,611PF,MN,NaN,TN,None,NaN,NaN
291,260PG,MN,NaN,Michoacán,MX,02/01/01,NaN
292,175PG,MN,NaN,Michoacán,MX,02/28/01,NaN
293,295PG,MN,NaN,Michoacán,MX,01/26/01,NaN


### Rename values in Report Country column

In [32]:
new_table.loc[new_table['Report Country'] == 'MX', 'Report Country'] = 'MEXICO'
new_table.loc[new_table['Report Country'].isnull(), 'Report Country'] = 'USA'
new_table

,Tag No.,Tag State,Tag Date,Report State,Report Country,Report Date,Miles
0,613PZ,CT,09/24/00,Michoacán,MEXICO,01/26/01,2222.0
1,486KF,ONT,09/12/99,Michoacán,MEXICO,01/07/01,2090.0
2,067GA,MI,08/20/00,Michoacán,MEXICO,01/07/01,1995.0
3,154JR,VA,09/25/99,MEXICO,MEXICO,03/09/01,1969.0
4,017MS,PA,09/01/99,Michoacán,MEXICO,12/01/00,1927.0
...,...,...,...,...,...,...,...
290,611PF,MN,NaN,TN,USA,NaN,NaN
291,260PG,MN,NaN,Michoacán,MEXICO,02/01/01,NaN
292,175PG,MN,NaN,Michoacán,MEXICO,02/28/01,NaN
293,295PG,MN,NaN,Michoacán,MEXICO,01/26/01,NaN


### Final check for unwanted spaces

In [33]:
# new_table.loc[new_table['Report State'].str.contains(r'\s', na=False), 'Report State'].value_counts()
# new_table.loc[new_table['Tag State'].str.contains(r'\s', na=False), 'Tag State'].value_counts()
# new_table.loc[new_table['Tag No.'].str.contains(r'\s', na=False), 'Tag No.'].value_counts()
# new_table.loc[new_table['Report Country'].str.contains(r'\s', na=False), 'Report Country'].value_counts()

###  Convert date columns to DateTime datatypes

In [34]:
new_table["Tag Date"]= pd.to_datetime(new_table["Tag Date"]) 
new_table["Report Date"]= pd.to_datetime(new_table["Report Date"]) 
new_table.dtypes

Tag No.                   object
Tag State                 object
Tag Date          datetime64[ns]
Report State              object
Report Country            object
Report Date       datetime64[ns]
Miles                    float64
dtype: object

In [35]:
del df2, df3, df4, df5

### Export cleaned data to csv.

In [36]:
new_table.to_csv("Monarchs2000.csv")